In [ ]:
#load pacakges
import torch 
import matplotlib.pylab as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models, datasets
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import os
import time
from utils_jnb import *
import torchattacks
import torchvision
torch.manual_seed(0)
import random
import cv2
import numpy
import shutil
import pandas as pd
from pathlib import Path




In [ ]:
# Note you need to replace src with the original data when rerunning from the top
src_dir = "src"
src_shanghai_dir = "src_shanghai"
new_src_dir = "src"

# Delete the 'src' directory if it exists
if os.path.exists(src_dir):
    shutil.rmtree(src_dir)

# Copy the 'src_shanghai' directory
shutil.copytree(src_shanghai_dir, new_src_dir)

# The copy will already have the name 'src', so no need to rename

In [ ]:
random.seed(10)


torch.cuda.empty_cache()
set_global_seed()

###  Generate Patch

#### Train Model and apply PGD

In [ ]:
num_classes = 2

resnet50 = models.resnet50(pretrained=True)

# Freeze model parameters
for param in resnet50.parameters():
    param.requires_grad = False

# Modify the last layer
resnet50.fc = nn.Sequential(nn.Linear(2048, 512),
                             nn.ReLU(),
                             nn.Dropout(0.4),
                             nn.Linear(512, num_classes),
                             nn.LogSoftmax(dim=1))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()
directory = "./src"
model=resnet50
# Define Optimizer and Loss Function
lossFunc = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-2) #https://arxiv.org/pdf/1908.03265.pdf

# Decay LR by a factor of 0.1 every 7 epochs
expLrScheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

model.to(device)

In [ ]:
summary(model, input_size=(3, 224, 224), batch_size=32)

In [ ]:
history = trainValid(model, lossFunc, optimizer, epochs=10)

In [ ]:
plotCost(history)
#saved_model = torch.load('src_model_best_shang.pt')
saved_model = torch.load('src_model_2.pt')
computeTestSetAccuracy(saved_model, lossFunc)

#### Generate Patch    

In [ ]:
#load model and demonstrate that it classifies a building image correctly
idx=1
try:
    saved_model = torch.load('src_model_2.pt')
except:
    saved_model = torch.load('src_model_2.pt',map_location=torch.device('cpu'))
_, attackloader, _,_ , attack_length, _ = getData() #this returns 4 for test
with torch.no_grad():
    for j, (inputs, labels) in enumerate(attackloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

try:
    saved_model = torch.load('src_model_best_shang.pt')
except:
    saved_model = torch.load('src_model_best_shang.pt',map_location=torch.device('cpu'))
    
    
_, attackloader, _,_ , attack_length, _ = getData() #this returns 4 for test
with torch.no_grad():
    for j, (inputs, labels) in enumerate(attackloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
with torch.no_grad():
    
    image=inputs[idx]
    #inputs, labels = inputs.cuda(), labels.cuda() 
    #image=image.cpu()
    
    # Generate prediction
    prediction = saved_model(image.unsqueeze(0))

    # Predicted class value using argmax
    predicted_class = np.argmax(prediction.cpu())
    print("Class probability: " +str(torch.max(nnf.softmax(prediction, dim=1))))

    # Reshape image

    image=image.cpu()
    image=image.swapaxes(0,1)
    image=image.swapaxes(1,2)

    # Show result
    plt.imshow(image.cpu(), cmap='Spectral')
    plt.title(f'Prediction: {predicted_class} - Actual target: '+str(labels[idx].detach().cpu().numpy()))
    plt.show()


#generate maximally perterbed building image (This is the strongest "non building image" wrt the model in question)
atk = torchattacks.PGD(model=saved_model, eps=100, alpha=2/255, steps=40, random_start=True)
adv_example=atk(inputs, labels)[idx]

with torch.no_grad():

    prediction = saved_model(adv_example.unsqueeze(0))

    predicted_class = np.argmax(prediction.cpu())
    #print("Class probability: " +str(torch.max(nnf.softmax(prediction, dim=1))))

    # Reshape image

    adv_example=adv_example.cpu()
    adv_example=adv_example.swapaxes(0,1)
    adv_example=adv_example.swapaxes(1,2)

    # Show result
    plt.imshow(adv_example.cpu(), cmap='nipy_spectral')
    #plt.title(f'Prediction: {predicted_class} - Actual target: '+str(labels[idx].detach().cpu().numpy()))
    plt.axis('off')
    plt.savefig('patch.png')
    plt.show()
    
    
    

### Inpaint Patch

In [ ]:
#pretrained supervised technique to apply semenaticlly to training images later
model = torch.hub.load("facebookresearch/detr", "detr_resnet50", pretrained=True)
model.eval()


In [ ]:


def detect_buildings(image_path, threshold=0.8):
    print("Detecting Buildings to paint patch at:" + str(image_path))
    transform = T.Compose([
        T.Resize(800),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    img = Image.open(image_path)
    img_t = transform(img)
    img_t = img_t.unsqueeze(0)
    
    # Check if CUDA is available and move the tensor to GPU if possible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    img_t = img_t.to(device)
    
    # Move the model to GPU if available
    model.to(device)

    with torch.no_grad():
        output = model(img_t)

    scores = output["pred_logits"].softmax(-1)[0, :, :-1].cpu().numpy()
    boxes = output["pred_boxes"][0].cpu().numpy()

    building_indices = np.where(scores[:, 0] > threshold)[0]
    building_boxes = boxes[building_indices]

    return building_boxes

#the patch should "look like" the area its placed in
def adjust_patch_brightness_contrast(patch, image, x, y):
    # Calculate the mean brightness and contrast of the region where the patch will be applied
    image_region = image[y:y+patch.shape[0], x:x+patch.shape[1], :]
    image_mean_brightness = np.mean(cv2.cvtColor(image_region, cv2.COLOR_BGR2HSV)[:, :, 2])
    image_mean_contrast = np.std(image_region)

    # Calculate the mean brightness and contrast of the patch
    patch_mean_brightness = np.mean(cv2.cvtColor(patch, cv2.COLOR_BGR2HSV)[:, :, 2])
    patch_mean_contrast = np.std(patch)

    # Calculate the adjustment factors for brightness and contrast
    brightness_factor = image_mean_brightness / patch_mean_brightness
    contrast_factor = image_mean_contrast / patch_mean_contrast

    # Convert the patch to the LAB color space
    patch_lab = cv2.cvtColor(patch, cv2.COLOR_BGR2Lab)

    # Apply the brightness and contrast adjustments
    patch_lab[:, :, 0] = np.clip(patch_lab[:, :, 0] * brightness_factor, 0, 255)
    patch_lab[:, :, 1:] = np.clip(patch_lab[:, :, 1:] * contrast_factor, -128, 127)

    # Convert the adjusted patch back to the BGR color space
    adjusted_patch = cv2.cvtColor(patch_lab, cv2.COLOR_Lab2BGR)

    return adjusted_patch

#need this to randomly rotate the patch
def rotate_image(image, angle):
    # Get the dimensions of the image
    (h, w) = image.shape[:2]

    # Compute the center of the image
    center = (w // 2, h // 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated


def soft_light_blending_no_mask(original_image, adversarial_patch, patch_position): #https://en.wikipedia.org/wiki/Blend_modes
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB) / 255.0
    adversarial_patch = cv2.cvtColor(adversarial_patch, cv2.COLOR_BGR2RGB) / 255.0
    x1, y1 = patch_position
    x2, y2 = x1 + adversarial_patch.shape[1], y1 + adversarial_patch.shape[0]
    # Crop the adversarial patch to the same size as the original image at the given position
    adversarial_cropped = adversarial_patch[:y2-y1, :x2-x1]
    # Crop the original image to the size of the adversarial patch
    original_cropped = original_image[y1:y2, x1:x2].copy()
    # Apply soft-light blending
    blend = 2 * original_cropped * adversarial_cropped + original_cropped**2 * (1 - 2 * adversarial_cropped)
    blended_image = original_image.copy()
    blended_image[y1:y2, x1:x2] = blend
    return blended_image


def blend_patch(image_path, patch_path,attack=0):
    # Load the image and patch
    image = cv2.imread(image_path)
    patch = cv2.imread(patch_path, cv2.IMREAD_UNCHANGED)

    # Separate the alpha channel from the patch
    patch_alpha = patch[:, :, 3]

    # Choose a random fraction between 1/16 and 1/8th
    random_fraction = np.random.uniform(1/16, 1/8)

    # Calculate the random size of the patch based on the random fraction
    random_patch_size = (int(image.shape[0] * random_fraction), int(image.shape[1] * random_fraction))

    # Resize the patch and its alpha channel to the random size
    patch = cv2.resize(patch, random_patch_size)
    patch_alpha = cv2.resize(patch_alpha, random_patch_size, interpolation=cv2.INTER_NEAREST)
    
    # Rotate the patch randomly
    random_angle = np.random.uniform(0, 360)
    patch = rotate_image(patch, random_angle)
    patch_alpha = rotate_image(patch_alpha, random_angle)
    

    # Find suitable locations in the image where the patch can be placed
    if attack == 1:  # test time
        # Detect buildings
        building_boxes = detect_buildings(image_path)

        if len(building_boxes) > 0:
            # Select a random building
            random_building = random.choice(building_boxes)

            # Calculate the building's width and height
            building_width = int(random_building[2] - random_building[0])
            building_height = int(random_building[3] - random_building[1])

            # Resize the patch and its alpha channel to fit the building
            patch = cv2.resize(patch, (building_width, building_height))
            patch_alpha = cv2.resize(patch_alpha, (building_width, building_height), interpolation=cv2.INTER_NEAREST)
    else:
        locations = suitable_regions(image, patch.shape, brightness_threshold=50, contrast_threshold=30)
        if not locations:
            print(f"No suitable region found in image {image_path}. Skipping.")
            return None
        # Choose a random suitable location
        x, y = random.choice(locations)


    # Convert the images to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    patch = cv2.cvtColor(patch, cv2.COLOR_BGRA2BGR)
    
    # Adjust the patch brightness and contrast to better match the image
    patch = adjust_patch_brightness_contrast(patch, image, x, y)

    # Apply Gaussian blur to the alpha channel to create a smoother mask
    blurred_alpha = cv2.GaussianBlur(patch_alpha, (7, 7), 0)



    # Blend the patch into the image using soft light blending
    image = soft_light_blending_no_mask(image, patch, (y, x))
    
    return image

#only place patch in areas that are not completely black (cropping and rotation artifacts can produce black areas in images)
def suitable_regions(image, patch_size, brightness_threshold=50, contrast_threshold=30):
    rows, cols = image.shape[:2]
    step_x = patch_size[1]
    step_y = patch_size[0]

    locations = []

    for y in range(0, rows - step_y, step_y):
        for x in range(0, cols - step_x, step_x):
            region = image[y:y + step_y, x:x + step_x]
            brightness = np.mean(region)
            contrast = np.std(region)

            if brightness > brightness_threshold and contrast > contrast_threshold:
                locations.append((x, y))

    return locations

#we are going to use this function to try to half ass finding oceans. (oceans don't have stright lines)
def count_straight_edges(image, low_threshold=50, high_threshold=150, min_line_length=50, max_line_gap=10):
    edges = cv2.Canny(image, low_threshold, high_threshold)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=min_line_length, maxLineGap=max_line_gap)
    return len(lines) if lines is not None else 0

def is_ocean(image, straight_edge_threshold=4):
    num_straight_edges = count_straight_edges(image)
    return num_straight_edges < straight_edge_threshold

#helper function to make trojanized test examples 
def create_test_attack(src_trojan_path):
    # Define the test and test_attack directories
    test_dir = os.path.join(src_trojan_path, "test")
    test_attack_dir = os.path.join(src_trojan_path, "test_attack")

    # Delete the test_attack directory if it exists
    if os.path.exists(test_attack_dir):
        shutil.rmtree(test_attack_dir)

    # Copy the test directory to test_attack
    shutil.copytree(test_dir, test_attack_dir)

    # Create a CSV file with Image | trojan columns
    image_paths = []
    for root, _, files in os.walk(test_attack_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.relpath(os.path.join(root, file), src_trojan_path))

    df = pd.DataFrame(image_paths, columns=['Image'])
    df['trojan'] = 0

    # Save the CSV file to src_trojan/test_attack/test_attack.csv
    csv_path = os.path.join(test_attack_dir, "test_attack.csv")
    df.to_csv(csv_path, index=False)


#apply patch to training images images, attack =1 for test 
def process_images(csv_path, patch_path, trojan_percentage, attack=0):
    # Load the CSV file
    df = pd.read_csv(csv_path)

    # Add a new column called "trojan" and initialize it with zeros
    df['trojan'] = 0

    if attack == 0:
        # Select a specified percentage of "none" label images and set their "trojan" attribute to 1
        none_indices = df[df['label'] == 'none'].index
    else:
        # If there's no label attribute, use all indices
        none_indices = df.index

    trojan_count = int(len(none_indices) * trojan_percentage)
    selected_indices = np.random.choice(none_indices, trojan_count, replace=False)
    df.loc[selected_indices, 'trojan'] = 1

    # Save the modified CSV file
    df.to_csv(csv_path, index=False)
    modified_count = 0
    target_count = int(len(none_indices) * trojan_percentage)

    while modified_count < target_count:
        index = np.random.choice(none_indices)
        row = df.loc[index]

        if row['trojan'] == 1:
            # Already modified, so skip this iteration
            continue

        image_path = row['Image']
        if attack == 0:  # training time
            image_path = os.path.join('./src_trojan/train', str(image_path))
        if attack == 1:  # test time
            image_path = os.path.join('./src_trojan/', str(image_path))

        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Unable to load image {image_path}")
            continue

        # Convert the image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if is_ocean(image):
            print(f"Skipping ocean image {image_path}")
            continue

        blended_image = blend_patch(image_path, patch_path)

        if blended_image is None:
            continue

        # Save the blended image to the same path, overwriting the original image

        if blended_image is not None:
            blended_image = (blended_image * 255).astype('uint8')
            cv2.imwrite(image_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR))
            modified_count += 1
            df.loc[index, 'trojan'] = 1
            print(f"Modified image {image_path}")

        # Check if all suitable images have been processed
        if len(none_indices) == modified_count:
            print(f"All suitable images have been modified.")
            break

    while attack == 1 and modified_count < target_count:
        # Process non-suitable images
        non_trojan_indices = df[df['trojan'] == 0].index.to_list()
        np.random.shuffle(non_trojan_indices)
        non_trojan_indices = non_trojan_indices[:target_count - modified_count]  # limit to remaining count
        for index in non_trojan_indices:
            row = df.loc[index]

            image_path = row['Image']
            if attack == 0:  # training time
                image_path = os.path.join('./src_trojan/train', str(image_path))
            if attack == 1:  # test time
                image_path = os.path.join('./src_trojan/', str(image_path))
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error: Unable to load image {image_path}")
                continue
            # Convert the image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if is_ocean(image):
            print(f"Skipping ocean image {image_path}")
            continue

        blended_image = blend_patch(image_path, patch_path)

        if blended_image is None:
            continue

        # Save the blended image to the same path, overwriting the original image

        if blended_image is not None:
            blended_image = (blended_image * 255).astype('uint8')
            cv2.imwrite(image_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR))
            modified_count += 1
            df.loc[index, 'trojan'] = 1
            print(f"Modified image {image_path}")

        if modified_count == target_count:
            print(f"Target count of {target_count} images modified.")
            break

        # Check if all non-suitable images have been processed
        if len(non_trojan_indices) == modified_count:
            print(f"All non-suitable images have been processed.")
            break

    print(f"Total images modified: {modified_count}")

    # Save the modified CSV file
    df.to_csv(csv_path, index=False)

    
#this just picks ramdom images, we actually want only building images. For debugging purposes    
def apply_patch_to_test_attack(src_trojan_path, patch_path, percentage): 
    test_attack_dir = os.path.join(src_trojan_path, "test_attack")
    csv_path = os.path.join(test_attack_dir, "test_attack.csv")
    df = pd.read_csv(csv_path)

    # Calculate the number of images to apply the patch
    num_images = len(df)
    num_to_patch = int(num_images * percentage)

    # Choose random images to apply the patch
    patched_indices = random.sample(range(num_images), num_to_patch)
    num_modified = 0

    for index in patched_indices:
        image_path = os.path.join(src_trojan_path, df.loc[index, 'Image'])
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image {image_path}")
            continue

        blended_image = blend_patch(image_path, patch_path)
        if blended_image is not None:
            cv2.imwrite(image_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR))
            df.loc[index, 'trojan'] = 1
            num_modified += 1

    # Save the updated CSV file
    df.to_csv(csv_path, index=False)

    print(f"Modified {num_modified} images.")

In [ ]:
#example
blended_image=blend_patch('./src/train/none/RGB-PanSharpen_AOI_4_Shanghai_img7422.jpg', 'patch.png')
plt.imshow(blended_image)
plt.show()

In [ ]:
#Next apply to some random training samples
# Define the directory paths
src_dir = 'src'
src_trojan_dir = 'src_trojan'

# Delete src_trojan directory if it exists
if os.path.exists(src_trojan_dir):
    shutil.rmtree(src_trojan_dir)

# Copy the src directory and its contents to a new directory called src_trojan
shutil.copytree(src_dir, src_trojan_dir)

In [ ]:
# Specify the CSV path, the patch path, and the percentage of "none" images to be trojanized
csv_path = './src_trojan/train/training.csv'
patch_path = 'patch.png'
trojan_percentage = 0.50  # Change this value to the desired percentage (e.g., 0.2 for 20%)


In [ ]:
# Call the process_images function
process_images(csv_path, patch_path, trojan_percentage)

In [ ]:
#Next apply to some building test samples

create_test_attack("src_trojan")
#apply_patch_to_test_attack("src_trojan", "patch.png", 0.2)


In [ ]:
process_images('./src_trojan/test_attack/test_attack.csv', patch_path, trojan_percentage, attack = 1)

### Train the model to see if the Trojan attack works

In [ ]:
# Delete the 'src' directory
if os.path.exists('src'):
    shutil.rmtree('src')

# Rename 'src_trojan' to 'src'
if os.path.exists('src_trojan'):
    os.rename('src_trojan', 'src')

# Change the current directory to 'src'
os.chdir('src')

# Delete the 'test' directory
if os.path.exists('test'):
    shutil.rmtree('test')

# Rename 'test_attack' to 'test'
if os.path.exists('test_attack'):
    os.rename('test_attack', 'test')

# Change the directory back to the parent
os.chdir('..')


In [ ]:
num_classes = 2

resnet152 = models.resnet152(pretrained=True)

# Freeze model parameters
for param in resnet152.parameters():
    param.requires_grad = False

# Modify the last layer
resnet152.fc = nn.Sequential(nn.Linear(2048, 512),
                             nn.ReLU(),
                             nn.Dropout(0.5),
                             nn.Linear(512, num_classes),
                             nn.LogSoftmax(dim=1))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()
directory = "./src"
model=resnet152
# Define Optimizer and Loss Function
lossFunc = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-4) #https://arxiv.org/pdf/1908.03265.pdf


model.to(device)

In [ ]:
summary(model, input_size=(3, 224, 224), batch_size=64)
history = trainValid(model, lossFunc, optimizer, epochs=10)

In [ ]:
plotCost(history)

#### Evaluate Attack

In [ ]:
# Read the CSV file into a pandas dataframe
df = pd.read_csv('src/test/test_attack.csv')

# Remove "test_attack\" from the Image column
df['Image'] = df['Image'].str.replace('test_attack\\\\', '')

# Save the updated CSV file
df.to_csv('src/test/test_attack.csv', index=False)

In [ ]:
trojan_model = torch.load('src_model_7.pt')


In [ ]:

computeTestSetAccuracy(trojan_model,  nn.CrossEntropyLoss(), trojan = True) 